In [1]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

import seaborn as sns
from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv(r'/Users/selinsevim/Desktop/breast_cancer_prediction_ai_model/data/raw/data.csv')

In [3]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [4]:
df.columns

Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst', 'Unnamed: 32'],
      dtype='object')

In [5]:
df.describe()

,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
count,5.690000e+02,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,0.0
mean,3.037183e+07,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,...,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946,NaN
std,1.250206e+08,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,...,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061,NaN
min,8.670000e+03,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,...,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040,NaN
25%,8.692180e+05,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,...,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460,NaN
50%,9.060240e+05,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,...,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040,NaN
75%,8.813129e+06,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,...,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080,NaN
max,9.113205e+08,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,...,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500,NaN


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [7]:
#report = ProfileReport(df)

In [8]:
#report

In [9]:
# Check if there are Null values in the dataframe
df.isna().sum()

id                           0
diagnosis                    0
radius_mean                  0
texture_mean                 0
perimeter_mean               0
area_mean                    0
smoothness_mean              0
compactness_mean             0
concavity_mean               0
concave points_mean          0
symmetry_mean                0
fractal_dimension_mean       0
radius_se                    0
texture_se                   0
perimeter_se                 0
area_se                      0
smoothness_se                0
compactness_se               0
concavity_se                 0
concave points_se            0
symmetry_se                  0
fractal_dimension_se         0
radius_worst                 0
texture_worst                0
perimeter_worst              0
area_worst                   0
smoothness_worst             0
compactness_worst            0
concavity_worst              0
concave points_worst         0
symmetry_worst               0
fractal_dimension_worst      0
Unnamed:

In [10]:
df = df.drop(columns = "Unnamed: 32", axis = 1) 

In [11]:
df.columns

Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst'],
      dtype='object')

In [12]:
df.isnull().any()

id                         False
diagnosis                  False
radius_mean                False
texture_mean               False
perimeter_mean             False
area_mean                  False
smoothness_mean            False
compactness_mean           False
concavity_mean             False
concave points_mean        False
symmetry_mean              False
fractal_dimension_mean     False
radius_se                  False
texture_se                 False
perimeter_se               False
area_se                    False
smoothness_se              False
compactness_se             False
concavity_se               False
concave points_se          False
symmetry_se                False
fractal_dimension_se       False
radius_worst               False
texture_worst              False
perimeter_worst            False
area_worst                 False
smoothness_worst           False
compactness_worst          False
concavity_worst            False
concave points_worst       False
symmetry_w

In [13]:
df["diagnosis"] = df["diagnosis"].map({'M':1, 'B':0})

In [14]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [63]:
# Map numeric diagnosis back to string labels for plotting
diagnosis_labels = df['diagnosis'].map({0: 'Benign', 1: 'Malignant'})

fig = px.histogram(x=diagnosis_labels, 
                   color=diagnosis_labels,
                   color_discrete_map={
        'Benign': 'rgb(50, 80, 168)',     
        'Malignant': '#ee8ef5'     
    })
fig.update_xaxes(categoryorder="array", categoryarray=['Benign', 'Malignant'])
fig.update_layout(
    title="Distribution of Diagnosis",
    xaxis_title="Diagnosis",
    yaxis_title="Count of Diagnosis Distribution"
)
fig.update_layout(
    width=600,
    height=400,
    font=dict(
        family="Arial, Courier New, monospace",  # font family
        size=12,                                 # font size (pixels)
        color="darkblue"                         # font color
    )
)

fig.update_layout(
    plot_bgcolor='white',
    xaxis=dict(
        showline=True,            # show axis line
        linecolor='darkgray',        # axis line color
        showgrid=True,            # show grid lines
        gridcolor='lightgray',    # grid line color
        gridwidth=1
    ),
    yaxis=dict(
        showline=True,
        linecolor='darkgray',
        showgrid=True,
        gridcolor='lightgray',
        gridwidth=1
    )
)

fig.show()

In [20]:
df.corr()['diagnosis'] > 0

id                          True
diagnosis                   True
radius_mean                 True
texture_mean                True
perimeter_mean              True
area_mean                   True
smoothness_mean             True
compactness_mean            True
concavity_mean              True
concave points_mean         True
symmetry_mean               True
fractal_dimension_mean     False
radius_se                   True
texture_se                 False
perimeter_se                True
area_se                     True
smoothness_se              False
compactness_se              True
concavity_se                True
concave points_se           True
symmetry_se                False
fractal_dimension_se        True
radius_worst                True
texture_worst               True
perimeter_worst             True
area_worst                  True
smoothness_worst            True
compactness_worst           True
concavity_worst             True
concave points_worst        True
symmetry_w

In [72]:
fig = px.scatter_matrix(df,
    dimensions=['radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst'],
    color="diagnosis", 
    color_continuous_scale="Purp")

fig.update_layout(
    title=dict(text='Breast Cancer Diagnosis'),
    width=2000,
    height=2000,
    font=dict(
        family="Arial, Courier New, monospace",  # font family
        size=12,                                 # font size (pixels)
        color="darkblue"                         # font color
    ),
    plot_bgcolor='white'
)

fig.show()

In [69]:
fig = px.scatter_matrix(df,
    dimensions=["radius_mean", "texture_mean", "perimeter_mean", "area_mean","smoothness_mean", "compactness_mean"],
    color="diagnosis", 
    color_continuous_scale="Purp")

fig.update_layout(
    title=dict(text='Breast Cancer Diagnosis'),
    width=1000,
    height=1000,
    font=dict(
        family="Arial, Courier New, monospace",  # font family
        size=12,                                 # font size (pixels)
        color="darkblue"                         # font color
    ),
    plot_bgcolor='white'
)

fig.show()

In [77]:
px.histogram(df, x="radius_worst", color="diagnosis", opacity=0.6, nbins=30)

In [79]:
px.histogram(df, x="area_worst", color="diagnosis", opacity=0.6, nbins=30)

In [ ]:
# Benign cases: Peak at lower values (left side)
# Malignant cases: Peak at higher values (right side)
# Minimal overlap in the middle

In [78]:
px.histogram(df, x="symmetry_mean", color="diagnosis", opacity=0.6, nbins=30)

In [ ]:
# Both groups: Similar peaks at similar locations
# Heavy overlap throughout the range
# Hard to distinguish the two colors

Clear Separation = Easy Classification:

When histograms show distinct peaks with minimal overlap
A simple threshold could classify most cases correctly
High accuracy expected from machine learning models

Heavy Overlap = Difficult Classification:

When both diagnosis groups have similar distributions
High error rates expected
Models will struggle to distinguish between classes

Threshold-Based Classification:
If you see clear separation (like area_worst), you could literally draw a vertical line:
If area_worst > 800: Predict Malignant
If area_worst ≤ 800: Predict Benign
This simple rule might achieve 85-90% accuracy!

Features with good separation → Keep these
good_features = ["area_worst", "radius_worst", "concavity_worst"]

Features with poor separation → Consider removing
weak_features = ["symmetry_mean", "fractal_dimension_mean"]

In [67]:
columns_to_plot = ['radius_mean', 'texture_mean', 'perimeter_mean']

for column in columns_to_plot:
    fig = px.histogram(df, x=df[column], color="diagnosis", marginal="box",
                       color_discrete_map={
                                0: 'rgb(50, 80, 168)',     
                                1: '#ee8ef5'})
    fig.update_layout(
    title=dict(text=f"Box plot of {column} by Diagnosis"),
    width=800,
    height=400
    )
    fig.update_layout(
    width=600,
    height=400,
    font=dict(
        family="Arial, Courier New, monospace",  # font family
        size=12,                                 # font size (pixels)
        color="darkblue"                         # font color
    )
    )

    fig.update_layout(
        plot_bgcolor='white',
        xaxis=dict(
            showline=True,            # show axis line
            linecolor='darkgray',        # axis line color
            showgrid=True,            # show grid lines
            gridcolor='lightgray',    # grid line color
            gridwidth=1
        ),
        yaxis=dict(
            showline=True,
            linecolor='darkgray',
            showgrid=True,
            gridcolor='lightgray',
            gridwidth=1
        )
    )
    fig.show()

In [66]:
columns_to_plot = ['radius_mean', 'texture_mean', 'perimeter_mean']

for column in columns_to_plot:
    fig = px.box(df, x="diagnosis", 
                 y=column, 
                 color="diagnosis", 
                 color_discrete_map={
                                0: 'rgb(50, 80, 168)',     
                                1: '#ee8ef5'}
    )
    fig.update_layout(
        title=dict(text=f"Box plot of {column} by Diagnosis"),
        width=800,
        height=400
    )
    fig.update_layout(
    width=600,
    height=400,
    font=dict(
        family="Arial, Courier New, monospace",  # font family
        size=12,                                 # font size (pixels)
        color="darkblue"                         # font color
    )
    )

    fig.update_layout(
        plot_bgcolor='white',
        xaxis=dict(
            showline=True,            # show axis line
            linecolor='darkgray',        # axis line color
            showgrid=True,            # show grid lines
            gridcolor='lightgray',    # grid line color
            gridwidth=1
        ),
        yaxis=dict(
            showline=True,
            linecolor='darkgray',
            showgrid=True,
            gridcolor='lightgray',
            gridwidth=1
        )
    )
    fig.show()

In [68]:
corr_matrix = df.corr(numeric_only=True)
fig = go.Figure(data=go.Heatmap(
    z=corr_matrix.values,
    x=corr_matrix.columns,
    y=corr_matrix.columns,
    colorscale='Purp',  
    text=corr_matrix.values,  # Add the correlation values as text
    texttemplate="%{z:.2f}",  # Format the text (2 decimal places)
    showscale=True
))
fig.update_layout(title='Correlation Heatmap')
fig.update_layout(
    width=1000,
    height=800,
    font=dict(
        family="Arial, Courier New, monospace",  # font family
        size=12,                                 # font size (pixels)
        color="darkblue"                         # font color
    )
)
fig.show()

In [49]:
df_numeric = df.copy()
target_corr = df_numeric.corr()['diagnosis'].drop('diagnosis').sort_values(ascending=False)

print("Features most correlated with diagnosis:")
print(target_corr.head(10))

print("Features negatively correlated with diagnosis:")
print(target_corr.tail(10))

Features most correlated with diagnosis:
concave points_worst    0.793566
perimeter_worst         0.782914
concave points_mean     0.776614
radius_worst            0.776454
perimeter_mean          0.742636
area_worst              0.733825
radius_mean             0.730029
area_mean               0.708984
concavity_mean          0.696360
concavity_worst         0.659610
Name: diagnosis, dtype: float64
Features negatively correlated with diagnosis:
symmetry_mean              0.330499
fractal_dimension_worst    0.323872
compactness_se             0.292999
concavity_se               0.253730
fractal_dimension_se       0.077972
id                         0.039769
symmetry_se               -0.006522
texture_se                -0.008303
fractal_dimension_mean    -0.012838
smoothness_se             -0.067016
Name: diagnosis, dtype: float64


In [61]:
# Find features with correlation > 0.9 (or <-0.9)
threshold = 0.9
corr_pairs = corr_matrix.abs().unstack().sort_values(ascending=False)

# Filter out self-correlations
high_corr_pairs = [(a, b, corr_matrix.loc[a, b]) for a, b in corr_pairs.index if a != b and corr_matrix.loc[a, b] > threshold]

# Show results
for a, b, corr_val in high_corr_pairs:
    print(f"{a} & {b} => correlation: {corr_val:.3f}")

radius_mean & perimeter_mean => correlation: 0.998
perimeter_mean & radius_mean => correlation: 0.998
perimeter_worst & radius_worst => correlation: 0.994
radius_worst & perimeter_worst => correlation: 0.994
radius_mean & area_mean => correlation: 0.987
area_mean & radius_mean => correlation: 0.987
perimeter_mean & area_mean => correlation: 0.987
area_mean & perimeter_mean => correlation: 0.987
radius_worst & area_worst => correlation: 0.984
area_worst & radius_worst => correlation: 0.984
area_worst & perimeter_worst => correlation: 0.978
perimeter_worst & area_worst => correlation: 0.978
radius_se & perimeter_se => correlation: 0.973
perimeter_se & radius_se => correlation: 0.973
perimeter_mean & perimeter_worst => correlation: 0.970
perimeter_worst & perimeter_mean => correlation: 0.970
radius_mean & radius_worst => correlation: 0.970
radius_worst & radius_mean => correlation: 0.970
perimeter_mean & radius_worst => correlation: 0.969
radius_worst & perimeter_mean => correlation: 0.96